In [34]:
from driver_manager import DriverManager
from employee_data_crawler import EmployeeDataCrawler
from time_off_calendar_crawler import TimeOffCalendarCrawler
from uploader import Uploader
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import TimeoutException
import os
import sys
import util

def username_password_input() -> (str, str):
    print('Client - Paycom Username:')
    username = input()
    print('Client - Paycom Password')
    password = input()
    return username, password

def terminate():
    driver.quit()

def get_session_ready() -> bool:
    username, password = username_password_input()
    login_result = driver_manager.login(username, password)
    if login_result == True:
        driver = driver_manager.driver_hand_over()
        return True
    else:
        return False

driver_manager = DriverManager()
driver = None
base_dir = ''
if getattr(sys, 'frozen', False):
    base_dir = os.path.dirname(sys.executable)
else:
    base_dir = os.path.abspath('')

is_session_ready = get_session_ready()
if is_session_ready == True:
    print("LOGIN SUCCESSFUL. SESSION IS READY.")
    driver = driver_manager.driver_hand_over()
else:
    print("ERROR. LOGIN FAILED. SESSION IS NOT READY.")
    exit()




temp test


2023-05-09 13:08:05,314 INFO Your selenium-driver-updater library is up to date. 
2023-05-09 13:08:18,457 INFO Current version of chromedriver: 113.0.5672.63 
2023-05-09 13:08:18,813 INFO Latest version of chromedriver: 113.0.5672.63 
2023-05-09 13:08:18,814 INFO Your existing chromedriver is up to date. current_version: 113.0.5672.63 latest_version: 113.0.5672.63 


Chrome User Data Path: C:\Users\HIS96347\PycharmProjects\paycomCrawler\chrome_user_data
Client - Paycom Username:


 sangjunlee


Client - Paycom Password


 payNeh@85246


logged in
LOGIN SUCCESSFUL. SESSION IS READY.


In [38]:
driver.get("https://www.paycomonline.net/v4/cl/web.php/employee/changes")
delay = 10
WebDriverWait(driver, delay).until(
    expected_conditions.presence_of_element_located((By.CLASS_NAME, 'css-vkr5jj')))
print("'Make Employee Changes' page loaded")

result = []


'Make Employee Changes' page loaded


In [39]:
import time
import traceback
from selenium.common.exceptions import StaleElementReferenceException


result = []


users = driver.find_elements(By.CLASS_NAME, 'css-vkr5jj')

employee_ids = []

driver.execute_script("return document.body.scrollHeight")

before_page_y_offset = driver.execute_script("return window.pageYOffset;")
not_hit_bottom = 1

while not_hit_bottom:
    
    try:
        employees = driver.find_elements(By.CSS_SELECTOR, 'div.css-gwrvcm')
        one_row_height = 81

        for each_employee in employees:

            result_dict = {}

            # one_row_height = each_employee.size['height']

            employee_id = each_employee.find_element(By.CSS_SELECTOR, 'span.css-20f01v').text
            if employee_id not in employee_ids:

                each_employee.find_element(By.CSS_SELECTOR, 'button.css-kb77mr').click()
                # driver.implicitly_wait(10)
                time.sleep(5)
                
                driver.execute_script("arguments[0].scrollIntoView();", driver.find_element(By.CSS_SELECTOR, 'div.css-927gzi'))
                time.sleep(5)

                ###
                temp_name = each_employee.find_element(By.CSS_SELECTOR, 'div.css-ewvkoh').find_element(By.CSS_SELECTOR, 'h1.jssEEFormsRewrite17').text
                driver.implicitly_wait(10)

                result_dict['name'] = util.name_formatter(temp_name)

                result_dict['EID'] = each_employee.find_element(By.CSS_SELECTOR, 'span.jssEEFormsRewrite11').text.replace('(','').replace(')','')
                
                result_dict['position'] = each_employee.find_element(By.CSS_SELECTOR, 'div.css-eix830').text
                
                

                time.sleep(5)
                phones_emails_managers = each_employee.find_elements(By.CSS_SELECTOR, 'div.css-1uf4nsy')
                driver.implicitly_wait(5)

                phones_emails_managers[0].find_elements(By.CSS_SELECTOR, 'button')
                driver.implicitly_wait(5)
                print("\tfound buttons")
                for index_no in range(3):
                    phones_emails_managers[index_no].click()
                    driver.implicitly_wait(10)

                    lines = driver.find_element(By.CSS_SELECTOR, 'div.css-djbxmn').text.splitlines()
                    driver.implicitly_wait(5)

                    key, key_prefix, value = "", "", ""
                    if index_no == 0:
                        key_prefix = "Phone Number"
                    if index_no == 1:
                        key_prefix = "Email"
                    if index_no == 2:
                        key_prefix = "Manager"
                    cnt = 0
                    for each in lines:
                        if index_no < 2:
                            if cnt % 2 == 0:
                                key = key_prefix + " - " + each
                            if cnt % 2 == 1:
                                value = each
                                result_dict[key] = value
                        else:
                            if cnt % 4 == 0:
                                key = key_prefix + " - " + each
                            if cnt % 4 == 2:
                                value = each
                                result_dict[key] = util.name_formatter(value)
                        cnt = cnt + 1
                left_tab_components = driver.find_element(By.CSS_SELECTOR, 'div.css-1kf24an').find_elements(By.CSS_SELECTOR, 'div.css-j486mh')
                for each in left_tab_components:
                    lines = each.text.splitlines()
                    key, value = "", ""
                    cnt = 0
                    for line in lines:
                        if cnt%4 == 0:
                            key = line
                        if cnt%4 == 1:
                            value = line
                        cnt = cnt + 1
                    result_dict[key] = util.name_formatter(value)

                employment_table = driver.find_element(By.CSS_SELECTOR, 'div.css-18hyvxw')
                contents = employment_table.find_elements(By.CSS_SELECTOR, 'div.css-j486mh')
                for each in contents:
                    str_lines = each.text.splitlines()
                    result_dict[str_lines[0]] = str_lines[1]

                result.append(result_dict)
                
                print(employee_id)
                employee_ids.append(employee_id)

                ###

        driver.execute_script("window.scrollBy(0, " + str(one_row_height * 10) + ")","")
        current_page_y_offset = driver.execute_script("return window.pageYOffset;")
        if current_page_y_offset == before_page_y_offset:
            not_hit_bottom = 0
        else:
            before_page_y_offset = current_page_y_offset
    
    except StaleElementReferenceException:
        # print(traceback.format_exc())
        print("StaleElementReferenceException: Reopening the page...")
        driver.get("https://www.paycomonline.net/v4/cl/web.php/employee/changes")
        delay = 10
        WebDriverWait(driver, delay).until(
            expected_conditions.presence_of_element_located((By.CLASS_NAME, 'css-vkr5jj')))
        print("'Make Employee Changes' page loaded")
   

	found buttons
(1227)
	found buttons
(0751)
	found buttons
(1508)
	found buttons
(0764)
	found buttons
(0841)
	found buttons
(0623)
	found buttons
(0731)
	found buttons
(1149)
	found buttons
(1038)
	found buttons
(1663)
	found buttons
(1424)
	found buttons
(0212)
	found buttons
(1088)
	found buttons
(1083)
	found buttons
(1354)
	found buttons
(0605)
	found buttons
(1629)
	found buttons
(1691)
	found buttons
(1665)
	found buttons
(1346)
	found buttons
(1646)
	found buttons
(1582)
	found buttons
(0722)
StaleElementReferenceException: Reopening the page...
'Make Employee Changes' page loaded
	found buttons
(1631)
	found buttons
(0972)
	found buttons
(1689)
	found buttons
(0497)
	found buttons
(0723)
	found buttons
(1656)
	found buttons
(1676)
	found buttons
(1617)
StaleElementReferenceException: Reopening the page...
'Make Employee Changes' page loaded
StaleElementReferenceException: Reopening the page...
'Make Employee Changes' page loaded
StaleElementReferenceException: Reopening the pa

TimeoutException: Message: 
Stacktrace:
Backtrace:
	GetHandleVerifier [0x008B8893+48451]
	(No symbol) [0x0084B8A1]
	(No symbol) [0x00755058]
	(No symbol) [0x00780467]
	(No symbol) [0x0078069B]
	(No symbol) [0x007ADD92]
	(No symbol) [0x0079A304]
	(No symbol) [0x007AC482]
	(No symbol) [0x0079A0B6]
	(No symbol) [0x00777E08]
	(No symbol) [0x00778F2D]
	GetHandleVerifier [0x00B18E3A+2540266]
	GetHandleVerifier [0x00B58959+2801161]
	GetHandleVerifier [0x00B5295C+2776588]
	GetHandleVerifier [0x00942280+612144]
	(No symbol) [0x00854F6C]
	(No symbol) [0x008511D8]
	(No symbol) [0x008512BB]
	(No symbol) [0x00844857]
	BaseThreadInitThunk [0x75CA00F9+25]
	RtlGetAppContainerNamedObjectPath [0x76F37BBE+286]
	RtlGetAppContainerNamedObjectPath [0x76F37B8E+238]


In [30]:
print(result)

[{'name': 'Masum Ahamed', 'position': 'SQA Analyst', 'Phone Number - Primary (Cell)': '+1 (313) 775-0589', 'Email - Work': 'masum.ahamed@emp.btisolutions.com', 'Email - Personal': 'masumahmed496@gmail.com', 'Manager - Primary': 'Kiwook Kang', 'Manager - Time Off': 'Kiwook Kang', 'Termination Date': '09/29/2021', 'Primary Supervisor': 'Andy Kang', 'Status': 'Terminated', 'Pay Type': 'Hourly', 'ESS Login': 'Masumahmed496@Gmail.Com', 'Primary Org': 'LG [53]', 'Manager': 'Kiwook Kang [DM02]', 'Division': 'Division-S [1-2]', 'Terminal': 'TG3', 'Schedule Group': 'N/A', 'Badge Number': 'N/A', 'Pay Frequency': 'Bi-Weekly', 'Most Recent Check': 'N/A', 'ESS Access Profile': 'Terminated ESS', 'Last 4 Digits of SSN': '[XXX-XX-2860]', 'Works-In State': 'Michigan', 'Lives-In State': 'Michigan', 'Exempt Status': 'Non-Exempt', 'Pending Time-Off Requests': 'N/A', 'On-Leave Start': 'N/A', 'On-Leave End': 'N/A', '401(k) Eligibility Date': 'N/A', 'Outstanding Checklists': '2', 'Document Group': 'N/A', 'Da